# Practical 5 - Dimensionality Reduction with PCA and t-SNE

Names: Pratik Dhameliya,Injamam Ul Karim,Hasan Marwan Mahmood   
Summer Term 2023   
Due Date: Tuesday, June 27, 2pm

In this practical we will implement dimensionality reduction with PCA, followed by t-SNE. Following is going to be the outline:

 1. Imports
 2. Loading data and related transformations 
 3. PCA without sklearn 
 4. PCA with sklearn and comparison with 3. 
 5. t-SNE with sklearn 

You will be plotting relevant results for viusalization and understanding as you go through the objectives. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(context='talk',style='white',palette='colorblind')
import pickle
import matplotlib
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy as sp

## Task 0: Load and normalize count data

This practical uses the data set from https://www.nature.com/articles/s41586-018-0654-5. This is single cell transcriptomics data from ~25,000 cells from the cortex. 

For each of these cells, the expression of several thousand genes was measured ```['counts']```. In the original study, the authors were interested in clustering the cells into types. 

We made a selection of 5000 cells and the 1000 most informative genes for run time reasons. We provide you with the original cell type labels determined by the authors for comparison ```['clusters']```.

The following function will apply some preprocessing steps that are standard for transcriptomics data. We normalize the data to bring columns to comparable sizes and log-transform them as they contain huge outliers.

In [ ]:
def lognormalize_counts(tasic_dict):

    # normalize and logtransform counts
    counts = tasic_dict['counts']
    libsizes = counts.sum(axis=1)
    CPM = counts / libsizes * 1e+6
    logCPM = np.log2(CPM + 1)
    tasic_dict['logCPM'] = np.array(logCPM)

    return tasic_dict

In [ ]:
tasic_1k = lognormalize_counts(pickle.load(open('/kaggle/input/tasicdata/tasic_subset_1kselected.pickle', 'rb')))

Have a look at ```['counts']```, ```['logPCM']``` and ```['clusters']``` to get a better understanding of the data. Plot a histogram of the cell type labels provided by  ```['clusters']```.

In [ ]:
# Explore data
# Explore 'counts', 'logPCM', and 'clusters' data
counts_data = tasic_1k['counts']
logPCM_data = tasic_1k['logCPM']
clusters_data = tasic_1k['clusters']
import scipy.sparse
variable = scipy.sparse.csc_matrix(counts_data)
data_type = type(variable).__name__
print("'counts':", variable)

In [ ]:
# Explore data
print('LogCPM values:', tasic_1k['logCPM'][:2, :4])
print('LogCPM shape', tasic_1k['logCPM'].shape)
print('counts values:', tasic_1k['counts'][:5, :8])
print('counts shape', tasic_1k['counts'].shape)
print('clusters values:', tasic_1k['clusters'][:5])
print('clusters shape:', tasic_1k['clusters'].shape)


In [ ]:
# Plotting
import matplotlib.pyplot as plt

plt.hist(clusters_data, bins=10)
plt.xlabel('Cell Type')
plt.ylabel('Frequency')
plt.title('Histogram of Cell Type Labels')
plt.show()


## Task 1: Linear dimensionality reduction with PCA

In this task, you will use Principal Component Analysis (PCA) to reduce the dimensionality of the dataset.

First, implement PCA "by hand". You can use eigenvalue/singular value decomposition from numpy/scipy but no `sklearn`-functions. Write a function that computes all possible principal components and returns them along with the fraction of variance they explain.

In [ ]:
def PCA_manual(data):
    '''
    Function that performs PCA on the input data

    input: (cells, genes)-shaped array of log transformed cell counts
    output:
        fraction_variance_explained: (genes,)-shaped array with the fraction of variance explained by the individual PCs
        principal_components: (genes, genes)-shaped array containing the principal components as columns
    '''
    ### NOTE: Make sure the function returns the PCs sorted by the fraction of variance explained! ###
    ###       (First column of principal_components should hold the PC with the highest variance   ###
    ###       explained -- fraction_variance_explained should also be sorted accordingly)          ###
    ###       Also remember to center your columns by substracting the mean of each column before  ###
    ###       you calculate the covariance matrix to get the principal components.                 ###

    # ---------------- INSERT CODE ----------------------
 # Center the data by subtracting the mean of each column
    centered_data = data - np.mean(data, axis=0)

    # Compute the covariance matrix
    covariance_matrix = np.cov(centered_data, rowvar=False)

    # Perform eigenvalue decomposition on the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # Sort the eigenvalues and eigenvectors in descending order of eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Compute the fraction of variance explained by each principal component
    fraction_variance_explained = sorted_eigenvalues / np.sum(sorted_eigenvalues)

    # Compute the principal components
    #principal_components = np.dot(centered_data, sorted_eigenvectors)
    principal_components = sorted_eigenvectors

    # ---------------- END CODE -------------------------

    return fraction_variance_explained, principal_components

In [ ]:
var_expl, PCs = PCA_manual(tasic_1k['logCPM'])

Before we explore the structure of the low-dimensional representation, we first want to know how much variance the first PCs explain: 

- Plot the fraction of variance explained by the `n`-th PC vs. `n`

- Plot the cumulative fraction of variance explained by the first `n` PCs with largest eigenvalue vs. `n`

From the latter plot you should be able to see how many PCs you need to keep to explain at least `x`% of the variance.

How many components do you need to keep to explain 50%, 75%, 90% and 99%, respectively? Indicate this in your plot.

In [ ]:
n_PCs = len(var_expl)
PC_ids = np.arange(1, n_PCs+1)

plt.figure(figsize=(14, 7))

plt.subplot(121)

# Plot the variance explained of the n-th PC vs. n
# ---------------- INSERT CODE ----------------------
plt.plot(PC_ids, var_expl, marker='o')
plt.xlabel('Principal Component')
plt.ylabel('Fraction of Variance Explained')
plt.title('Fraction of Variance Explained by Principal Components')
# ---------------- END CODE -------------------------

plt.subplot(122)

# Plot the cumulative variance explained for the n PCs with highest variance explained vs. n
# Indicate how many components you need to keep to explain 50%, 75%, 90% and 99% in the plot.

# ---------------- INSERT CODE ----------------------

cumulative_var_expl = np.cumsum(var_expl)
plt.plot(PC_ids, cumulative_var_expl, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Fraction of Variance Explained')
plt.title('Cumulative Fraction of Variance Explained by Principal Components')

# Indicate how many components you need to keep to explain 50%, 75%, 90% and 99% in the plot
var_expl_thresholds = [0.5, 0.75, 0.9, 0.99]
for threshold in var_expl_thresholds:
    n_components = np.argmax(cumulative_var_expl >= threshold) + 1
    plt.axvline(x=n_components, color='red', linestyle='--')
    plt.text(n_components+1, threshold, f'{threshold*100}%')

plt.tight_layout()
plt.show()


# ---------------- END CODE -------------------------

In [ ]:
print(np.abs(np.cumsum(var_expl)[:10]-0.5))
print('Pcs Amount We need for 50%', np.abs(np.cumsum(var_expl) - 0.5).argmin()+1)
print('Pcs Amount We need for 75%', np.abs(np.cumsum(var_expl) - 0.75).argmin()+1)
print('Pcs Amount We need for 90%', np.abs(np.cumsum(var_expl) - 0.9).argmin()+1)
print('Pcs Amount We need for 99%', np.abs(np.cumsum(var_expl) - 0.99).argmin()+1)

YOUR ANSWER HERE

Write a function to select the `n` PCs needed to explain at least `x`% of the variance and use this function to extract as many PCs as are needed to explain 75% of the variance. 

In [ ]:
def select_PCs(
    variance_explained, principal_components, percent_variance=None):
    '''Function that selects the first n principal components necessary to explain x% of the variance
    input:
        variance_explained: amount of variance explained by the individual PCs
        principal_components: contains the principal components as columns
        percent_variance: fraction of the variance, the all PCs that are kept explain
    output:
        variance_explained_kept: individual amount of variance explained for the remaining PCs
        principal_components_kept: remaining principal components, shape (genes,n_PCs_kept)
    '''

    # ---------------- INSERT CODE ----------------------

    if percent_variance is not None:
        # Compute cumulative sum of variance explained
        cumulative_variance = np.cumsum(variance_explained)

        # Find the index where cumulative variance exceeds or equals the desired percentage
        n_PCs_kept = np.argmax(cumulative_variance >= percent_variance) + 1
        print(f'Number of PCs kept: {n_PCs_kept}') # this must be 41

        # Select the corresponding variance explained and principal components
        variance_explained_kept = variance_explained[:n_PCs_kept]
        principal_components_kept = principal_components[:, :n_PCs_kept]
    else:
        # If percent_variance is not provided, keep all principal components
        variance_explained_kept = variance_explained
        principal_components_kept = principal_components

    
    
    # ---------------- END CODE -------------------------

    return variance_explained_kept, principal_components_kept

In [ ]:
_, PCs75 = select_PCs(var_expl, PCs, percent_variance=0.75)
print(PCs75.shape)

To compute the representation of the data in this lower dimensional representation, write a function that compute the PC scores for each cell, i.e. that projects the original data matrix on the low-dimensional subspace provided by the first `n` PCs:

In [ ]:
def compute_PCA_scores(data, principal_components):
    '''Function that returns the PC scores for each data point
    input:
        data                 --- (cells, genes)-shaped array of log transformed cell counts
        principal_components --- contains the principal components as columns
    output:
        pc_scores            --- (cells, n_PCs_kept)-shaped array of PC scores
    '''

    # ---------------- INSERT CODE ----------------------

    # Center the data by subtracting the mean along each gene dimension
    centered_data = data - np.mean(data, axis=0)

    # Compute the PC scores by projecting the centered data onto the principal components
    pc_scores = np.dot(centered_data, principal_components)

    
    # ---------------- END CODE -------------------------

    return pc_scores

In [ ]:
tasic_1k['PCA_75'] = compute_PCA_scores(tasic_1k['logCPM'], PCs75)

Visualize the top 5 PCs as a pairwise scatterplot. Use one subplot for each pair of components.

Use the colors provided in `tasic_1k['clusterColors']` and the cluster information in `tasic_1k['clusters']` to color each data point according to its original cluster identity.

The colors indicate the family of the cell type:

- greenish colors: excitatory neurons
- orange colors: somatostatin positive interneurons
- pinkish colors: VIP-postive interneurons
- reddish colors: parvalbumin positive interneurons
- dark colors: non-neurons (glia etc)

What do you observe?

In [ ]:
def plot_PCs(data_transformed, color_per_datapoint):
    '''Function that plots the scores of the 10 pairs of the top 5 PCs against each other.
        inputs:
            data_transformed    -- (cells, n_PCs_kept)-shaped array of PC scores
            color_per_datapoint -- (cells,)-shaped array of color strings, one color for each cell
    '''

    # ---------------- INSERT CODE ----------------------

    n_components =  5 #data_transformed.shape[1]
    
    fig, axes = plt.subplots(n_components, n_components, figsize=(15, 15))
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    
    for i in range(n_components):
        for j in range(n_components):
            if  i != j:
                ax = axes[i, j]
                ax.scatter(data_transformed[:, i], data_transformed[:, j], c=color_per_datapoint)
                ax.set_xlabel(f'PC {i+1}')
                ax.set_ylabel(f'PC {j+1}')
    
    plt.show()    
    # ---------------- END CODE -------------------------

In [ ]:
color_per_datapoint = tasic_1k['clusterColors'][tasic_1k['clusters']]
plot_PCs(tasic_1k['PCA_75'], color_per_datapoint)

YOUR ANSWER HERE

## Task 2: Comparison with PCA implemented by sklearn

Use the PCA implementation of sklearn to check whether your PCA implementation is correct and obtain some insights into numerical precision of the algorithms underlying PCA implementations. Note that the sklearn implementation of PCA switches the dimensions of the matrix, so you will have to transpose your principal components matrix to get the same output as in the manual implementation.

In [ ]:
def PCA_sklearn(data):
    '''
    Function that performs PCA on the input data, using sklearn

    input: (cells, genes)-shaped array of log transformed cell counts
    output:
        fraction_variance_explained: (genes,)-shaped array with the fraction of variance explained by the individual PCs
        principal_components: (genes, genes)-shaped array containing the principal components as columns
    '''

    # ---------------- INSERT CODE ----------------------
    pca = PCA()
    pca.fit(data)
    fraction_variance_explained = pca.explained_variance_ratio_
    principal_components = pca.components_.T
    # ---------------- END CODE -------------------------

    return fraction_variance_explained, principal_components

In [ ]:
# do sklearn-PCA on selected genes
var_expl_sklearn, PCs_sklearn = PCA_sklearn(tasic_1k['logCPM'])
# select components as before
_, PCs_sklearn75 = select_PCs(var_expl_sklearn, PCs_sklearn, 0.75)
# get PC scores
PCA_75_sklearn = compute_PCA_scores(tasic_1k['logCPM'], PCs_sklearn75)

To see if your manual PCA yielded the same PC weights as the sklearn PCA, we can just take the two matrices of principal components and plot their entries against each other. (Note: This again assumes they are sorted by variance explained and the order of dimensions in your weight matrix compared to the sklearn weight matrix is the same (change if necessary).)

Use the following plot to compare the results to your own implementation (here plotting the weights of the first 100 PCs against each other). What do you observe?

In [ ]:
n_evs_to_compare = 100

plt.figure(figsize=(10, 5))
plt.scatter(PCs_sklearn[:, :n_evs_to_compare].flatten(),
            PCs[:, :n_evs_to_compare].flatten(), s=5, alpha=0.1)
plt.plot([-.7, .7], [-.7, .7], ':', c='tab:gray', label='same sign', alpha=0.3)
plt.plot([-.7, .7], [.7, -.7], '--', c='tab:gray', label='sign flipped', alpha=0.3)
plt.legend()
plt.xlabel('sklearn PCA weight')
plt.ylabel('manual PCA weight')
plt.title('Weights of the first %u manual PCs against the sklearn ones' % (n_evs_to_compare))

sns.despine()

YOUR ANSWER HERE

Additional reading about the sign of PCs: https://stats.stackexchange.com/questions/88880/does-the-sign-of-scores-or-of-loadings-in-pca-or-fa-have-a-meaning-may-i-revers

## Task 3: Nonlinear dimensionality reduction with t-SNE

In this task, you will use the nonlinear dimensionality reduction technique tSNE and look at visualizations of the data set. Plot the result of default t-SNE with the original cluster colors. For this and the following tasks, use the PCs explaining 75% of the variance ```PCA_75_sklearn``` you computed above.

In [ ]:
def plot_tsne(tsne_results, clusters=tasic_1k['clusters'], labels=['']):
    '''Plotting function for tsne results, creates one or multiple plots of tSNE-transformed data.
       If the clustering is the original one (default), original cluster colors will be used. Otherwise,
       colors will be a random permutation.

    input:
        tsne_results: (n, 2)-shaped array containing tSNE-transformed data or list of such arrays
                      (output of the fit_transform function of sklearn tSNE)
        clusters: (n,)-shaped array containing cluster labels or list of such arrays
        labels: optional, list of titles for the subplots
    '''

    if type(tsne_results) == list:  # make sure we can do both single and multiple plots and are flexible regarding input
        num_plots = len(tsne_results)
    else:
        num_plots = 1
        tsne_results = [tsne_results]
    if type(clusters) == list:
        num_clusters = len(clusters)
        num_plots = num_plots * num_clusters
        tsne_results = tsne_results * num_clusters
    else:
        clusters = [clusters] * num_plots

    if len(labels) == 1:
        labels = labels * num_plots

    n_clusters = len(np.unique(clusters))      # ensure a long enough color list even if we plot more than
    n_colors = len(tasic_1k['clusterColors'])  # the original number of clusters
    if n_clusters > n_colors:
        n_extra_colors = n_clusters - n_colors
        colors = np.concatenate((tasic_1k['clusterColors'], tasic_1k['clusterColors'][:n_extra_colors]))
    else:
        colors = tasic_1k['clusterColors']

    fig, ax = plt.subplots(num_plots, 1, figsize=(10, num_plots*10))
    if num_plots == 1:
        if not np.all(tasic_1k['clusters'] == clusters[0]):
            current_colors = np.random.permutation(colors)
        else:
            current_colors = colors
        ax.scatter(tsne_results[0][:, 0], tsne_results[0][:, 1], s=1, color=current_colors[clusters[0]])
        ax.set_title(labels[0])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_axis_off()
    else:
        for i in range(num_plots):
            if not np.all(tasic_1k['clusters'] == clusters[i]):
                current_colors = np.random.permutation(colors)
            else:
                current_colors = colors
            ax[i].scatter(tsne_results[i][:, 0], tsne_results[i][:, 1], s=1, color=current_colors[clusters[i]])
            ax[i].set_title(labels[i])
            ax[i].set_xticks([])
            ax[i].set_yticks([])
            ax[i].set_axis_off()

Run the following cells to set the random seed/random state, run tSNE and plot the results.

In [ ]:
# fit TSNE
tsne_default = TSNE(random_state=1)
tsne_results = tsne_default.fit_transform(PCA_75_sklearn)

In [ ]:
# Plotting

original_clusters = tasic_1k['clusters']
plot_tsne(tsne_results, original_clusters, labels=['default t-SNE'])

t-SNE has one main parameter called perplexity, which trades of local and global structure. Its default value is 30. Run the tSNE with some other perplexity values (e.g. 5, 100), plot the results next to each other and explain what you observe. In particular, compare with the PCA plot above.

In [ ]:
# try different perplexities

# ---------------- INSERT CODE ----------------------
tsne_5 = TSNE(random_state=1, perplexity=5.)
tsne_results_5 = tsne_5.fit_transform(PCA_75_sklearn)
tsne_20 = TSNE(random_state=1, perplexity=20.)
tsne_results_20 = tsne_20.fit_transform(PCA_75_sklearn)
tsne_70= TSNE(random_state=1, perplexity=70.)
tsne_results_70 = tsne_70.fit_transform(PCA_75_sklearn)
tsne_90 = TSNE(random_state=1, perplexity=90.)
tsne_results_90 = tsne_90.fit_transform(PCA_75_sklearn)
tsne_200 = TSNE(random_state=1, perplexity=200.)
tsne_results_200 = tsne_200.fit_transform(PCA_75_sklearn)
tsne_100 = TSNE(random_state=1, perplexity=100.)
tsne_results_100 = tsne_100.fit_transform(PCA_75_sklearn)
# ---------------- END CODE -------------------------

In [ ]:
# plot results

# ---------------- INSERT CODE ----------------------
plot_tsne(tsne_results_5, original_clusters, labels=['t-SNE perplexity=5'])
plt.show()
plot_tsne(tsne_results_20, original_clusters, labels=['t-SNE perplexity=20'])
plt.show()
plot_tsne(tsne_results_70, original_clusters, labels=['t-SNE perplexity=70'])
plt.show()
plot_tsne(tsne_results_90, original_clusters, labels=['t-SNE perplexity=90'])
plt.show()
plot_tsne(tsne_results_100, original_clusters, labels=['t-SNE perplexity=100'])
plt.show()
plot_tsne(tsne_results_200, original_clusters, labels=['t-SNE perplexity=200'])
plt.show()
# ---------------- END CODE -------------------------

YOUR ANSWER HERE

Check out https://distill.pub/2016/misread-tsne/. There's a nice tool that let's you play with t-SNE paramters and visualize the consequences.